In [ ]:
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class ChopStick():
    def __init__(self, number):
        self.number = number  + 1        
        self.user = -1      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user):         # used for synchronization
        with self.lock:
            while self.taken == True:
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("Filósofo[%s] tomó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

    def drop(self, user):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1
            self.taken = False
            sys.stdout.write("Filósofo[%s] dejó el palillo[%s]\n" % (user, self.number))
            self.lock.notifyAll()

class Philosopher (threading.Thread):
    def __init__(self, number, left, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("Filósofo[%s] PENSANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number)    
            time.sleep(1)     
            self.right.take(self.number)    
            time.sleep(1)                
            sys.stdout.write("Filósofo[%s] COMIENDO\n" % (self.number))
            self.right.drop(self.number)    
            time.sleep(1)
            self.left.drop(self.number)     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------Filósofo[%s] terminó de COMER--------------\n" % self.number)

def dinner():
    n = int(input('Dime cuántos Filósofos : '))
    c = [ChopStick(i) for i in range(n)]
    semaforo = Semaphore(n-1)
    sys.stdout.write("\n")
    p = [Philosopher(i, c[i], c[(i+1)%n], semaforo) for i in range(n)]

    for i in range(n):
        p[i].start()
        
    for j in range(n):
        p[i].join()
    
dinner()

In [4]:
import threading
import time
import multiprocessing as mp
import sys
import random as rd

In [14]:
class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class puente():
    def __init__(self, number):
        self.number = number  + 1        
        self.user = -1      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == True:
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("Automóvil[%s] tomó el puente, [%s]\n" % (user, sentido))
            self.lock.notifyAll()

    def drop(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1
            self.taken = False
            sys.stdout.write("Automóvil[%s] dejó el puente [%s]\n" % (user, sentido))
            self.lock.notifyAll()
            

class AutomovilIzq (threading.Thread):
    def __init__(self, number, left, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.left = left
        #self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("<- Automóvil[%s] ESPERANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number, 'izquierdo')    
            time.sleep(1)     
            #self.right.take(self.number)    
            #time.sleep(1)                
            sys.stdout.write("<- Automóvil[%s] PASAND0\n" % (self.number))
            #self.right.drop(self.number)    
            #time.sleep(1)
            self.left.drop(self.number, 'izquierdo')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- <-Automóvil[%s] terminó de PASAR--------------\n" % (self.number))
        
class AutomovilDer (threading.Thread):
    def __init__(self, number, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        #self.left = left
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        for i in range(1):
            sys.stdout.write("-> Automóvil[%s] ESPERANDO\n" % (self.number))
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.right.take(self.number, 'dereho')    
            time.sleep(1)                 
            sys.stdout.write("-> Automóvil[%s] PASAND0\n" % (self.number))
            self.right.drop(self.number, 'derecho')     
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
        sys.stdout.write("---------------- -> Automóvil[%s] terminó de PASAR--------------\n" % self.number)
 
def transit():
    n = int(input('Dime cuántos Autos : '))
    izq = rd.randint(1,n/2+1)
    der = n - izq
    print('derechos :',der, 'izquierdos :', izq)
    c = [puente(i) for i in range(n)]
    semaforo = Semaphore(n-1)
    sys.stdout.write("\n")
    AIzq = [AutomovilIzq(i, c[i],  semaforo) for i in range(izq)]
    ADer = [AutomovilDer(i, c[i],  semaforo) for i in range(der)]

    for i in range(n):
        AIzq[i].start()
        ADer[i].start()
        
    for j in range(n):
        AIzq[i].join()
        ADer[i].join()
    
transit()

Dime cuántos Autos :  8


derechos : 7 izquierdos : 1

<- Automóvil[1] ESPERANDO
-> Automóvil[1] ESPERANDO


IndexError: list index out of range

Automóvil[1] tomó el puente, [izquierdo]
<- Automóvil[1] PASAND0
Automóvil[1] dejó el puente [izquierdo]
Automóvil[1] tomó el puente, [dereho]
---------------- <-Automóvil[1] terminó de PASAR--------------
-> Automóvil[1] PASAND0
Automóvil[1] dejó el puente [derecho]
---------------- -> Automóvil[1] terminó de PASAR--------------


In [ ]:
### ULTIMO FER

class Semaphore():
    def __init__(self, initial):
        self.lock = threading.Condition() # *
        self.value = initial

    def up(self):
        with self.lock:
            self.value += 1
            self.lock.notify() # *

    def down(self):
        with self.lock:
            while self.value == 0:
                self.lock.wait() # *
            self.value -= 1
            
class Puente():
    def __init__(self, number):
        self.number = number  + 1        
        self.user = -1      
        self.lock = threading.Condition()
        self.taken = False

    def take(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == True:
                self.lock.wait()
            self.user = user
            self.taken = True
            sys.stdout.write("Carro[%s, %s] pasa por el puente\n" % (sentido, user))
            self.lock.notifyAll()

    def drop(self, user, sentido):         # used for synchronization
        with self.lock:
            while self.taken == False:
                self.lock.wait()
            self.user = -1
            self.taken = False
            sys.stdout.write("-------------Carro[%s, %s] pasó por el puente----------\n" % (sentido, user))
            self.lock.notifyAll()
            
class Automovil_izq(threading.Thread):
    def __init__(self, number, left, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.left = left
        self.semaforo = semaforo
    
    def run(self):
        while True:
            sys.stdout.write("Carro sentido izquierdo quiere pasar\n" )
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.left.take(self.number, 'izquierdo')        
            time.sleep(1)                
            self.left.drop(self.number, 'izquierdo')  
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)
            
class Automovil_der(threading.Thread):
    def __init__(self, number, right, semaforo):
        threading.Thread.__init__(self)
        self.number = number + 1           
        self.right = right
        self.semaforo = semaforo
    
    def run(self):
        while True:
            sys.stdout.write("Carro sentido derecho quiere pasar\n")
            time.sleep(1)
            self.semaforo.down()              
            time.sleep(1)  
            self.right.take(self.number, 'derecho')        
            time.sleep(1)                
            self.right.drop(self.number, 'derecho')  
            time.sleep(1)
            self.semaforo.up()             
            #time.sleep(1)

def transito():
    n_izq = 5
    n_der = 5
    c = [Puente(i) for i in range(n_izq + n_der)]
    
    semaforo = Semaphore(n_izq + n_der -1)
    sys.stdout.write("\n")
    p_izq = [Automovil_izq(i, c[i], semaforo) for i in range(n_izq)]
    p_der = [Automovil_der(i, c[i], semaforo) for i in range(n_der)]

    for i in range(n_izq):
        p_izq[i].start()
    for i in range(n_der):
        p_der[i].start()
        
    for i in range(n_izq):
        p_izq[i].join()
    for i in range(n_der):
        p_der[i].join()
    
transito()